In [1]:
!pip install ydata-synthetic==1.1.0


In [9]:
from ydata_synthetic.synthesizers.regular import RegularSynthesizer
from ydata_synthetic.synthesizers import ModelParameters, TrainParameters
import pandas as pd
import io
import os
import requests
import numpy as np
from sklearn import metrics
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

#Load data and define the data processor parameters
df = pd.read_csv('https://raw.githubusercontent.com/mahayasa/gan-hybrid-sampling-customer-churn/main/data/telco_new.csv')


df_numerical_features = df.select_dtypes(exclude='object')
df_categorical_features = df.select_dtypes(include='object')

num_cols = list(df_numerical_features)
cat_cols = list(df_categorical_features)


In [10]:
train_data = df.loc[ df['Churn']==1 ].copy()

train_data

,Call Failure,Complains,Subscription Length,Charge Amount,Seconds of Use,Frequency of use,Frequency of SMS,Distinct Called Numbers,Age Group,Tariff Plan,Status,Age,Customer Value,FN,FP,Churn
16,0,0,37,0,875,14,0,11,2,1,2,25,40.005,36.0045,60.0000,1
18,0,0,37,0,0,0,0,0,2,1,2,25,0.000,0.0000,60.0000,1
22,23,1,33,0,955,47,16,17,2,1,2,25,117.090,105.3810,61.7090,1
24,13,1,36,1,5818,98,26,24,2,1,1,25,383.220,344.8980,88.3220,1
26,9,0,35,0,2990,41,9,16,3,1,2,30,157.240,141.5160,65.7240,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3126,14,1,37,0,3295,47,16,18,3,1,2,30,197.680,177.9120,69.7680,1
3127,14,1,38,0,2573,50,41,33,4,1,2,45,168.075,151.2675,66.8075,1
3128,5,0,38,0,438,8,7,4,3,1,2,30,45.840,41.2560,60.0000,1
3131,5,1,38,0,933,13,16,6,4,1,2,45,63.650,57.2850,60.0000,1


In [12]:


#Defining the training parameters
noise_dim = 128
dim = 128
batch_size = 500

log_step = 100
epochs = 100
learning_rate = [5e-4, 3e-3]
beta_1 = 0.5
beta_2 = 0.9
models_dir = '../cache'

gan_args = ModelParameters(batch_size=batch_size,
                           lr=learning_rate,
                           betas=(beta_1, beta_2),
                           noise_dim=noise_dim,
                           layers_dim=dim)

train_args = TrainParameters(epochs=epochs,
                             sample_interval=log_step)

synth = RegularSynthesizer(modelname='wgangp', model_parameters=gan_args, n_critic=2)
synth.fit(train_data, train_args, num_cols, cat_cols)

synth.save('insurance_wgangp_model.pkl')



/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
  3%|▎         | 3/100 [00:01<00:43,  2.21it/s]

Epoch: 0 | disc_loss: 245.3671112060547 | gen_loss: 0.014343547634780407
Epoch: 1 | disc_loss: 15.127110481262207 | gen_loss: 0.007560547906905413
Epoch: 2 | disc_loss: 6.86680269241333 | gen_loss: 0.03360883146524429


  5%|▌         | 5/100 [00:01<00:24,  3.83it/s]

Epoch: 3 | disc_loss: 1.9449427127838135 | gen_loss: 0.039054207503795624
Epoch: 4 | disc_loss: 0.6136632561683655 | gen_loss: 0.05305004492402077
Epoch: 5 | disc_loss: 0.3846469223499298 | gen_loss: 0.06361541152000427


  9%|▉         | 9/100 [00:02<00:13,  6.82it/s]

...

Epoch: 95 | disc_loss: 0.7972860932350159 | gen_loss: -0.09595638513565063
Epoch: 96 | disc_loss: 0.24372099339962006 | gen_loss: 0.036312829703092575


 99%|█████████▉| 99/100 [00:11<00:00,  7.62it/s]

Epoch: 97 | disc_loss: 1.4044551849365234 | gen_loss: -0.05888459086418152
Epoch: 98 | disc_loss: 0.31632357835769653 | gen_loss: -0.05107199028134346


100%|██████████| 100/100 [00:11<00:00,  8.71it/s]


Epoch: 99 | disc_loss: 0.6639707088470459 | gen_loss: -0.05389029160141945


In [13]:
#########################################################
#    Loading and sampling from a trained synthesizer    #
#########################################################
synth = RegularSynthesizer.load('insurance_wgangp_model.pkl')
synth_data = synth.sample(2655)

Synthetic data generation: 100%|██████████| 6/6 [00:00<00:00, 111.94it/s]


In [14]:
synth_data[synth_data['Churn']==1]

,Call Failure,Complains,Subscription Length,Charge Amount,Seconds of Use,Frequency of use,Frequency of SMS,Distinct Called Numbers,Age Group,Tariff Plan,Status,Age,Customer Value,FN,FP,Churn
0,8,0,46,0,2225,9,-12,3,3,0,1,25,-180.001556,424.376221,23.796503,1
1,7,0,35,0,1873,9,-5,1,3,0,1,24,-140.377853,369.856934,29.046095,1
2,5,0,42,0,2037,10,-4,5,3,0,1,24,-142.639221,374.827423,34.735008,1
3,13,0,49,0,2432,7,-15,1,3,0,1,25,-185.962479,380.965271,31.670506,1
4,8,0,38,0,1905,8,0,3,3,0,1,24,-80.149513,414.630249,30.343218,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2994,10,0,39,0,2210,2,11,6,3,0,1,26,-214.765167,312.113007,30.045546,1
2995,12,0,43,0,1616,1,-4,5,3,0,1,25,-92.473946,334.790222,30.942635,1
2996,9,0,45,0,3042,18,-8,1,3,0,1,27,-215.097427,343.517700,23.202791,1
2998,7,0,47,0,2002,7,3,-1,3,0,1,27,-77.065971,368.365723,32.933384,1


In [15]:
#concat original data and gan data
data_concat = pd.concat([df, synth_data])
# combine data churn and not churn
data=pd.concat([df, data_concat])

In [ ]:
data

,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,feature_11,feature_12,feature_13,feature_14,feature_15,labels
0,-0.276515,-0.424429,1.344997,-0.012283,0.076230,1.076648,0.182198,3,0,1,0,0,0,0,10,2,1
1,0.853573,0.150991,0.503892,-0.979179,-0.569351,-0.411453,-0.251940,4,1,2,0,1,0,0,0,3,0
2,0.947747,-0.173832,1.825628,-0.703478,0.076230,-0.411453,-0.251940,6,1,2,0,0,0,0,5,3,0
3,0.853573,-0.381404,0.984523,-0.039464,-0.569351,-0.411453,-0.251940,4,0,2,0,1,0,0,5,3,0
4,1.324443,1.590527,-1.178318,-0.097711,-0.246560,-0.411453,-0.251940,0,1,1,0,0,0,0,8,3,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26495,-2.391307,2.674451,-0.759365,1.783020,0.583692,0.578560,1.224344,1,0,0,0,0,0,1,6,1,1
26496,-2.864046,3.207475,-0.434721,2.369443,-0.252530,0.107089,-1.190178,2,0,0,0,0,0,0,4,1,1
26497,-2.603470,3.293620,-0.660011,2.204988,0.188471,0.236847,-1.431355,1,0,0,0,0,0,1,5,1,1
26498,-2.332877,1.821177,-0.782332,1.930105,0.486426,0.608122,1.736317,1,0,0,0,0,0,0,6,1,1


In [16]:
# -*- coding: utf-8 -*-
"""
Created on Mon Jun  1 17:31:40 2020

@author: manav

Modifed on 23 AUG 2022

by mahayasa adiputra
"""

import numpy as np
import pandas as pd
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import f1_score
from sklearn.metrics import auc
import matplotlib.pyplot as plt
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix , accuracy_score
from sklearn.metrics import roc_curve, roc_auc_score
import sklearn.metrics as mt
from imblearn.under_sampling import EditedNearestNeighbours
from imblearn.under_sampling import TomekLinks
from imblearn.under_sampling import NeighbourhoodCleaningRule
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import KNeighborsClassifier
from numpy import mean
from numpy import std
from sklearn.metrics import make_scorer
from imblearn.metrics import specificity_score
import math
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import GradientBoostingClassifier
import time

start1=time.time()

X=data.drop(['Churn'],axis=1)
y=data["Churn"]

#enn = EditedNearestNeighbours(n_neighbors=3)
#X, y = enn.fit_resample(X, y)
#ncr = NeighbourhoodCleaningRule(n_neighbors=5, kind_sel='all')
#X, y = ncr.fit_resample(X, y)
#tomek_links = TomekLinks()
#X, y = tomek_links.fit_resample(X, y)

#kfold cross validation
cv = KFold(n_splits=5, random_state=1, shuffle=True)
end1 = time.time()
print("The time of execution of preprocess:",
      (end1-start1), "s")

The time of execution of preprocess: 0.0021445751190185547 s


In [17]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [21]:
# decision tree
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.tree import DecisionTreeClassifier
model = DecisionTreeClassifier()
# define grid
balance1 = [{0:10,1:100},{0:20,1:100},{0:30,1:100},{0:40,1:100},{0:50,1:100},{0:60,1:100},{0:70,1:100},{0:80,1:100},{0:90,1:100},{0:100,1:100}]
balance = [{0:100,1:10},{0:100,1:20},{0:100,1:30},{0:100,1:40},{0:100,1:50},{0:100,1:60},{0:100,1:70},{0:100,1:80},{0:100,1:90},{0:100,1:100}]
param_grid = dict(class_weight=balance)
# define evaluation procedure
cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=1)
# define grid search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=cv, scoring='roc_auc')
# execute the grid search
grid_result = grid.fit(X, y)
# report the best configuration
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
# report all configurations
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

# Create a DataFrame with the results
results_df = pd.DataFrame(grid_result.cv_results_)
# Save the DataFrame to an Excel file
results_df.to_excel('wgangp-tn-insurance-dt-cs-auc.xlsx', index=False)


# Define the source file path (in Colab)
source_file = '/content/wgangp-tn-insurance-dt-cs-auc.xlsx'

# Define the destination folder path in Google Drive
destination_folder = "/content/drive/MyDrive/experiment/insurance"

# Copy the file to the destination
!cp $source_file $destination_folder

Best: 0.908109 using {'class_weight': {0: 100, 1: 20}}
0.907335 (0.006980) with: {'class_weight': {0: 100, 1: 10}}
0.908109 (0.005896) with: {'class_weight': {0: 100, 1: 20}}
0.907159 (0.005735) with: {'class_weight': {0: 100, 1: 30}}
0.907325 (0.006304) with: {'class_weight': {0: 100, 1: 40}}
0.906120 (0.007055) with: {'class_weight': {0: 100, 1: 50}}
0.904474 (0.005614) with: {'class_weight': {0: 100, 1: 60}}
0.906672 (0.007991) with: {'class_weight': {0: 100, 1: 70}}
0.905504 (0.006931) with: {'class_weight': {0: 100, 1: 80}}
0.903478 (0.006773) with: {'class_weight': {0: 100, 1: 90}}
0.904578 (0.008293) with: {'class_weight': {0: 100, 1: 100}}


In [23]:
from sklearn.metrics import f1_score, make_scorer
f1 = make_scorer(f1_score , average='macro')
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=cv, scoring=f1)
# execute the grid search
grid_result = grid.fit(X, y)
# report the best configuration
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
# report all configurations
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

# Create a DataFrame with the results
results_df = pd.DataFrame(grid_result.cv_results_)
# Save the DataFrame to an Excel file
results_df.to_excel('wgangp-tn-insurance-dt-cs-f1.xlsx', index=False)

# Define the source file path (in Colab)
source_file = '/content/wgangp-tn-insurance-dt-cs-f1.xlsx'

# Define the destination folder path in Google Drive
destination_folder = "/content/drive/MyDrive/experiment/insurance"

# Copy the file to the destination
!cp $source_file $destination_folder

/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/backend/fork_exec.py:38: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid = os.fork()


Best: 0.901735 using {'class_weight': {0: 100, 1: 20}}
0.901394 (0.006925) with: {'class_weight': {0: 100, 1: 10}}
0.901735 (0.005904) with: {'class_weight': {0: 100, 1: 20}}
0.899717 (0.004406) with: {'class_weight': {0: 100, 1: 30}}
0.900067 (0.005606) with: {'class_weight': {0: 100, 1: 40}}
0.899262 (0.005634) with: {'class_weight': {0: 100, 1: 50}}
0.900892 (0.005625) with: {'class_weight': {0: 100, 1: 60}}
0.901672 (0.007250) with: {'class_weight': {0: 100, 1: 70}}
0.900066 (0.007109) with: {'class_weight': {0: 100, 1: 80}}
0.900627 (0.004738) with: {'class_weight': {0: 100, 1: 90}}
0.900600 (0.006317) with: {'class_weight': {0: 100, 1: 100}}


In [24]:
from imblearn.metrics import geometric_mean_score
gm_scorer = make_scorer(geometric_mean_score, greater_is_better=True, average='binary')

grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=cv, scoring=gm_scorer)
# execute the grid search
grid_result = grid.fit(X, y)
# report the best configuration
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
# report all configurations
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))
# Create a DataFrame with the results
results_df = pd.DataFrame(grid_result.cv_results_)
# Save the DataFrame to an Excel file
results_df.to_excel('wgangp-tn-insurance-dt-cs-gmean.xlsx', index=False)

# Define the source file path (in Colab)
source_file = '/content/wgangp-tn-insurance-dt-cs-gmean.xlsx'

# Define the destination folder path in Google Drive
destination_folder = "/content/drive/MyDrive/experiment/insurance"

# Copy the file to the destination
!cp $source_file $destination_folder


Best: 0.900428 using {'class_weight': {0: 100, 1: 10}}
0.900428 (0.006661) with: {'class_weight': {0: 100, 1: 10}}
0.900406 (0.007354) with: {'class_weight': {0: 100, 1: 20}}
0.898187 (0.005494) with: {'class_weight': {0: 100, 1: 30}}
0.898282 (0.007259) with: {'class_weight': {0: 100, 1: 40}}
0.896345 (0.006034) with: {'class_weight': {0: 100, 1: 50}}
0.899131 (0.007236) with: {'class_weight': {0: 100, 1: 60}}
0.898311 (0.008090) with: {'class_weight': {0: 100, 1: 70}}
0.896971 (0.008429) with: {'class_weight': {0: 100, 1: 80}}
0.898288 (0.007472) with: {'class_weight': {0: 100, 1: 90}}
0.897279 (0.004996) with: {'class_weight': {0: 100, 1: 100}}


In [25]:
# LR
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
# define grid
balance1 = [{0:10,1:100},{0:20,1:100},{0:30,1:100},{0:40,1:100},{0:50,1:100},{0:60,1:100},{0:70,1:100},{0:80,1:100},{0:90,1:100},{0:100,1:100}]
balance = [{0:100,1:10},{0:100,1:20},{0:100,1:30},{0:100,1:40},{0:100,1:50},{0:100,1:60},{0:100,1:70},{0:100,1:80},{0:100,1:90},{0:100,1:100}]
param_grid = dict(class_weight=balance)
# define evaluation procedure
cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=1)
# define grid search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=cv, scoring='roc_auc')
# execute the grid search
grid_result = grid.fit(X, y)
# report the best configuration
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
# report all configurations
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

# Create a DataFrame with the results
results_df = pd.DataFrame(grid_result.cv_results_)
# Save the DataFrame to an Excel file
results_df.to_excel('wgangp-tn-insurance-lr-cs-auc.xlsx', index=False)

# Define the source file path (in Colab)
source_file = '/content/wgangp-tn-insurance-lr-cs-auc.xlsx'

# Define the destination folder path in Google Drive
destination_folder = "/content/drive/MyDrive/experiment/insurance"

# Copy the file to the destination
!cp $source_file $destination_folder

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Best: 0.916361 using {'class_weight': {0: 100, 1: 90}}
0.914220 (0.006757) with: {'class_weight': {0: 100, 1: 10}}
0.915119 (0.006540) with: {'class_weight': {0: 100, 1: 20}}
0.915895 (0.006304) with: {'class_weight': {0: 100, 1: 30}}
0.915994 (0.006339) with: {'class_weight': {0: 100, 1: 40}}
0.916313 (0.006368) with: {'class_weight': {0: 100, 1: 50}}
0.916199 (0.006309) with: {'class_weight': {0: 100, 1: 60}}
0.916134 (0.006110) with: {'class_weight': {0: 100, 1: 70}}
0.916051 (0.006286) with: {'class_weight': {0: 100, 1: 80}}
0.916361 (0.006063) with: {'class_weight': {0: 100, 1: 90}}
0.916213 (0.006138) with: {'class_weight': {0: 100, 1: 100}}


In [26]:
from sklearn.metrics import f1_score, make_scorer
f1 = make_scorer(f1_score , average='macro')
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=cv, scoring=f1)
# execute the grid search
grid_result = grid.fit(X, y)
# report the best configuration
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
# report all configurations
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

# Create a DataFrame with the results
results_df = pd.DataFrame(grid_result.cv_results_)
# Save the DataFrame to an Excel file
results_df.to_excel('wgangp-insurance-lr-cs-f1.xlsx', index=False)


# Define the source file path (in Colab)
source_file = '/content/wgangp-insurance-lr-cs-f1.xlsx'

# Define the destination folder path in Google Drive
destination_folder = "/content/drive/MyDrive/experiment/insurance"

# Copy the file to the destination
!cp $source_file $destination_folder

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Best: 0.836676 using {'class_weight': {0: 100, 1: 100}}
0.524673 (0.017380) with: {'class_weight': {0: 100, 1: 10}}
0.705396 (0.008499) with: {'class_weight': {0: 100, 1: 20}}
0.777116 (0.009654) with: {'class_weight': {0: 100, 1: 30}}
0.808518 (0.007386) with: {'class_weight': {0: 100, 1: 40}}
0.823371 (0.006934) with: {'class_weight': {0: 100, 1: 50}}
0.831066 (0.009203) with: {'class_weight': {0: 100, 1: 60}}
0.834894 (0.009998) with: {'class_weight': {0: 100, 1: 70}}
0.835697 (0.009260) with: {'class_weight': {0: 100, 1: 80}}
0.835248 (0.008701) with: {'class_weight': {0: 100, 1: 90}}
0.836676 (0.008910) with: {'class_weight': {0: 100, 1: 100}}


In [27]:
from imblearn.metrics import geometric_mean_score
gm_scorer = make_scorer(geometric_mean_score, greater_is_better=True, average='binary')
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=cv, scoring=gm_scorer)
# execute the grid search
grid_result = grid.fit(X, y)
# report the best configuration
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
# report all configurations
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

# Create a DataFrame with the results
results_df = pd.DataFrame(grid_result.cv_results_)
# Save the DataFrame to an Excel file
results_df.to_excel('wgangp-insurance-lr-cs-gmean.xlsx', index=False)


# Define the source file path (in Colab)
source_file = '/content/wgangp-insurance-lr-cs-gmean.xlsx'

# Define the destination folder path in Google Drive
destination_folder = "/content/drive/MyDrive/experiment/insurance"

# Copy the file to the destination
!cp $source_file $destination_folder

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Best: 0.829169 using {'class_weight': {0: 100, 1: 100}}
0.384431 (0.027782) with: {'class_weight': {0: 100, 1: 10}}
0.642562 (0.011715) with: {'class_weight': {0: 100, 1: 20}}
0.741396 (0.011959) with: {'class_weight': {0: 100, 1: 30}}
0.785576 (0.008580) with: {'class_weight': {0: 100, 1: 40}}
0.806041 (0.007467) with: {'class_weight': {0: 100, 1: 50}}
0.816830 (0.009785) with: {'class_weight': {0: 100, 1: 60}}
0.822826 (0.010668) with: {'class_weight': {0: 100, 1: 70}}
0.824996 (0.009847) with: {'class_weight': {0: 100, 1: 80}}
0.826020 (0.009474) with: {'class_weight': {0: 100, 1: 90}}
0.829169 (0.009730) with: {'class_weight': {0: 100, 1: 100}}


In [28]:
#SVM
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.svm import SVC
model = SVC()
# define grid
balance1 = [{0:10,1:100},{0:20,1:100},{0:30,1:100},{0:40,1:100},{0:50,1:100},{0:60,1:100},{0:70,1:100},{0:80,1:100},{0:90,1:100},{0:100,1:100}]
balance = [{0:100,1:10},{0:100,1:20},{0:100,1:30},{0:100,1:40},{0:100,1:50},{0:100,1:60},{0:100,1:70},{0:100,1:80},{0:100,1:90},{0:100,1:100}]
param_grid = dict(class_weight=balance)
# define evaluation procedure
cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=1)
# define grid search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=cv, scoring='roc_auc')
# execute the grid search
grid_result = grid.fit(X, y)
# report the best configuration
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
# report all configurations
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

# Create a DataFrame with the results
results_df = pd.DataFrame(grid_result.cv_results_)
# Save the DataFrame to an Excel file
results_df.to_excel('wgangp-tn-insurance-svm-cs-auc.xlsx', index=False)

# Define the source file path (in Colab)
source_file = '/content/wgangp-tn-insurance-svm-cs-auc.xlsx'

# Define the destination folder path in Google Drive
destination_folder = "/content/drive/MyDrive/experiment/insurance"

# Copy the file to the destination
!cp $source_file $destination_folder

Best: 0.925104 using {'class_weight': {0: 100, 1: 80}}
0.834392 (0.017267) with: {'class_weight': {0: 100, 1: 10}}
0.893369 (0.004979) with: {'class_weight': {0: 100, 1: 20}}
0.912707 (0.004787) with: {'class_weight': {0: 100, 1: 30}}
0.918858 (0.004383) with: {'class_weight': {0: 100, 1: 40}}
0.920988 (0.003859) with: {'class_weight': {0: 100, 1: 50}}
0.922807 (0.004202) with: {'class_weight': {0: 100, 1: 60}}
0.924493 (0.004022) with: {'class_weight': {0: 100, 1: 70}}
0.925104 (0.004269) with: {'class_weight': {0: 100, 1: 80}}
0.924549 (0.004667) with: {'class_weight': {0: 100, 1: 90}}
0.923133 (0.004709) with: {'class_weight': {0: 100, 1: 100}}


In [29]:
from sklearn.metrics import f1_score, make_scorer
f1 = make_scorer(f1_score , average='macro')
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=cv, scoring=f1)
# execute the grid search
grid_result = grid.fit(X, y)
# report the best configuration
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
# report all configurations
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

# Create a DataFrame with the results
results_df = pd.DataFrame(grid_result.cv_results_)
# Save the DataFrame to an Excel file
results_df.to_excel('wgangp-tn-insurance-svm-cs-f1.xlsx', index=False)

# Define the source file path (in Colab)
source_file = '/content/wgangp-tn-insurance-svm-cs-f1.xlsx'

# Define the destination folder path in Google Drive
destination_folder = "/content/drive/MyDrive/experiment/insurance"

# Copy the file to the destination
!cp $source_file $destination_folder

Best: 0.824688 using {'class_weight': {0: 100, 1: 40}}
0.384228 (0.000100) with: {'class_weight': {0: 100, 1: 10}}
0.721830 (0.010764) with: {'class_weight': {0: 100, 1: 20}}
0.824251 (0.009318) with: {'class_weight': {0: 100, 1: 30}}
0.824688 (0.009030) with: {'class_weight': {0: 100, 1: 40}}
0.824582 (0.009038) with: {'class_weight': {0: 100, 1: 50}}
0.824582 (0.009038) with: {'class_weight': {0: 100, 1: 60}}
0.824582 (0.009038) with: {'class_weight': {0: 100, 1: 70}}
0.824582 (0.009038) with: {'class_weight': {0: 100, 1: 80}}
0.824582 (0.009038) with: {'class_weight': {0: 100, 1: 90}}
0.824582 (0.009038) with: {'class_weight': {0: 100, 1: 100}}


In [30]:
from imblearn.metrics import geometric_mean_score
gm_scorer = make_scorer(geometric_mean_score, greater_is_better=True, average='binary')
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=cv, scoring=gm_scorer)
# execute the grid search
grid_result = grid.fit(X, y)
# report the best configuration
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
# report all configurations
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

# Create a DataFrame with the results
results_df = pd.DataFrame(grid_result.cv_results_)
# Save the DataFrame to an Excel file
results_df.to_excel('wgangp-tn-insurance-svm-cs-gmean.xlsx', index=False)


# Define the source file path (in Colab)
source_file = '/content/wgangp-tn-insurance-svm-cs-gmean.xlsx'

# Define the destination folder path in Google Drive
destination_folder = "/content/drive/MyDrive/experiment/insurance"

# Copy the file to the destination
!cp $source_file $destination_folder

Best: 0.809964 using {'class_weight': {0: 100, 1: 40}}
0.000000 (0.000000) with: {'class_weight': {0: 100, 1: 10}}
0.662392 (0.014530) with: {'class_weight': {0: 100, 1: 20}}
0.808948 (0.010175) with: {'class_weight': {0: 100, 1: 30}}
0.809964 (0.009897) with: {'class_weight': {0: 100, 1: 40}}
0.809887 (0.009890) with: {'class_weight': {0: 100, 1: 50}}
0.809887 (0.009890) with: {'class_weight': {0: 100, 1: 60}}
0.809887 (0.009890) with: {'class_weight': {0: 100, 1: 70}}
0.809887 (0.009890) with: {'class_weight': {0: 100, 1: 80}}
0.809887 (0.009890) with: {'class_weight': {0: 100, 1: 90}}
0.809887 (0.009890) with: {'class_weight': {0: 100, 1: 100}}
